# [구글 AI 기반 서비스 개발] 개인 미니 프로젝트
* **이름:** 이주혁
* **선택 모델:** Google Gemini-2.5-flash
* **프로젝트 목표:** 과제 안내 PDF의 내용을 학습하여, 과제에 대한 질문에 답변하는 RAG 챗봇 데모를 개발합니다.

In [2]:
# --- 1. 필요한 라이브러리 설치 ---
# 이 셀은 최초 한 번만 실행하면 됩니다.
!pip install --upgrade langchain langchain-openai langchain-google-genai langchain-text-splitters pypdf chromadb python-dotenv

# --- 2. 필요한 모든 모듈 Import ---
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# .env 파일에서 API 키 로드
load_dotenv()

True

In [ ]:
# 

PDF_FILE_PATH = "2025 10 (공유) 강원대 중간 고사 내용.pdf"
DB_PERSIST_DIRECTORY = "my_vector_db"

def load_and_split_documents(file_path):
    print(f"'{file_path}' 파일에서 PDF를 로드합니다...")
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_documents = text_splitter.split_documents(documents)
    print(f"✅ 문서 로드 및 분할 완료. (총 {len(split_documents)}개 청크)")
    return split_documents

# DB 폴더가 없으면 문서를 새로 로드하고, 있으면 건너뜁니다.
if not os.path.exists(DB_PERSIST_DIRECTORY):
    split_docs = load_and_split_documents(PDF_FILE_PATH)
else:
    split_docs = None # DB가 이미 있으므로 새로 로드할 필요 없음
    print("PDF 로드 및 분할 건너뛰기 (기존 DB 사용)")

PDF 로드 및 분할 건너뛰기 (기존 DB 사용)


In [19]:
embeddings = OpenAIEmbeddings()

if not os.path.exists(DB_PERSIST_DIRECTORY):
    print(f"DB를 새로 생성합니다.")
    vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=DB_PERSIST_DIRECTORY
    )
    print("DB 생성이 완료되었습니다.")
else:
    print(f"기존 DB를 불러옵니다. (경로: {DB_PERSIST_DIRECTORY})")
    vectorstore = Chroma(
        persist_directory=DB_PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    print("DB 로딩이 완료되었습니다.")

# DB에서 검색 기능을 수행할 Retriever를 준비
retriever = vectorstore.as_retriever()

기존 DB를 불러옵니다. (경로: my_vector_db)
DB 로딩이 완료되었습니다.


In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", streaming=True)
print(f"LLM 준비 완료: {llm.model}")

LLM 준비 완료: models/gemini-2.5-flash


In [22]:
system_prompt = """
당신은 주어진 내용을 바탕으로 사용자의 질문에 답변하는 친절한 AI 어시스턴트입니다.
이전 대화 내용을 참고하여 질문에 대한 답변을 생성하세요.
주어진 내용에서만 답변을 찾아야 하며, 내용을 지어내서는 안 됩니다.

내용:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

print("프롬프트 템플릿 준비 완료")

프롬프트 템플릿 준비 완료


In [23]:
# langchain 파이프라인을 이용한 RAG 체인 생성
rag_chain = (
    RunnablePassthrough.assign(context=lambda x: retriever.invoke(x["question"]))
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG 체인 생성 완료")

RAG 체인 생성 완료


In [24]:
# 대화 기록을 저장할 리스트 (메모리)
chat_history = []

def ask_chatbot(query):
    """
    질문을 받아 챗봇을 실행하고, 스트리밍으로 답변을 출력하며,
    대화 기록을 업데이트하는 함수
    """
    
    print("답변: ", end="", flush=True)
    
    # RAG 체인 실행
    response_stream = rag_chain.stream({
        "question": query,
        "chat_history": chat_history
    })
    
    full_answer = ""
    for chunk in response_stream:
        print(chunk, end="", flush=True)
        full_answer += chunk
    
    # 대화 기록 업데이트
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=full_answer))
    
    print("\n" + "-"*50)

print("대화 메모리 및 실행 함수 준비 완료")

대화 메모리 및 실행 함수 준비 완료


In [25]:
ask_chatbot("과제 제출일이 언제까지지???")

답변: 과제 제출 기한은 **2025년 11월 4일 (화요일)**까지입니다.
--------------------------------------------------


In [26]:
ask_chatbot("제출할 내용은 뭐가 있어???  300자 이내로 말해줘")

답변: 제출할 내용은 총 3가지입니다.
1.  **3분 데모 발표 영상** (mp4 등 일반적인 동영상 형식)
2.  **프로젝트 소스코드** (공개 GitHub Repository 링크, API Key 등 중요 정보가 들어가지 않도록 주의)
3.  **교수에게 하고 싶은 질문** (구글 폼을 통해 제출)
--------------------------------------------------


In [27]:
ask_chatbot("이전에 내가 뭘 물어봤었지??? 500자 이내로 말해줄래??")

답변: 이전에 사용자님께서 질문하셨던 내용은 다음과 같습니다:

1.  **과제 제출일이 언제까지인지** 궁금해하셨습니다.
2.  **제출할 내용이 무엇인지** 300자 이내로 알려달라고 요청하셨습니다.
--------------------------------------------------
